In [1]:
import datetime
import dill
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from zipfile import ZipFile
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from pandarallel import pandarallel
import warnings
import collections
import pickle
from scipy.sparse import csr_matrix
from scipy import sparse
warnings.simplefilter('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 20)

pandarallel.initialize(progress_bar=True, nb_workers=20)

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## Загрузим данные

In [7]:
# train упорядочен хронологически
# timespent: время залипания юзера на айтем в минутах (от 0 до 60)
# reaction: (1) - лайк, (-1) - дизлайк
train = pd.read_parquet('train.parquet.gzip')
train = train.reset_index().rename(columns={"index": "time"})
# train = train[train.timespent > 0]

print(f"Уникальных юзеров в interactions: {train['user_id'].nunique():_}")
print(f"Уникальных айтемов в interactions: {train['item_id'].nunique():_}")

train

Уникальных юзеров в interactions: 1_000_183
Уникальных айтемов в interactions: 227_606


,time,user_id,item_id,timespent,reaction
0,0,707536,67950,0,0
1,1,707536,151002,0,0
2,2,707536,134736,0,0
3,3,707536,196151,0,0
4,4,707536,94182,0,0
...,...,...,...,...,...
144440010,144440010,849764,80910,0,0
144440011,144440011,993316,132328,0,0
144440012,144440012,993316,186701,0,0
144440013,144440013,666981,81857,0,0


In [3]:
# в items_meta для каждого item_id его автор и эмбеддинг содержания
items_meta = pd.read_parquet('items_meta.parquet.gzip')
items_meta = items_meta[items_meta.item_id.isin(train.item_id.unique())]
items_meta[[f"feat_{i}" for i in range(312)]] = items_meta.embeddings.parallel_apply(pd.Series)
print("len rows: ", items_meta.shape[0])
print("Unique items: ", items_meta.item_id.nunique())
print("Unique writers: ", items_meta.source_id.nunique())
print("emb shape: ", items_meta.iloc[0].embeddings.shape)
items_meta.head()

len rows:  227606
Unique items:  227606
Unique writers:  24438
emb shape:  (312,)


,item_id,source_id,embeddings,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,feat_41,feat_42,feat_43,feat_44,feat_45,feat_46,feat_47,feat_48,feat_49,feat_50,feat_51,feat_52,feat_53,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,feat_94,feat_95,feat_96,feat_97,feat_98,feat_99,feat_100,feat_101,feat_102,feat_103,feat_104,feat_105,feat_106,feat_107,feat_108,feat_109,feat_110,feat_111,feat_112,feat_113,feat_114,feat_115,feat_116,feat_117,feat_118,feat_119,feat_120,feat_121,feat_122,feat_123,feat_124,feat_125,feat_126,feat_127,feat_128,feat_129,feat_130,feat_131,feat_132,feat_133,feat_134,feat_135,feat_136,feat_137,feat_138,feat_139,feat_140,feat_141,feat_142,feat_143,feat_144,feat_145,feat_146,feat_147,feat_148,feat_149,feat_150,feat_151,feat_152,feat_153,feat_154,feat_155,feat_156,feat_157,feat_158,feat_159,feat_160,feat_161,feat_162,feat_163,feat_164,feat_165,feat_166,feat_167,feat_168,feat_169,feat_170,feat_171,feat_172,feat_173,feat_174,feat_175,feat_176,feat_177,feat_178,feat_179,feat_180,feat_181,feat_182,feat_183,feat_184,feat_185,feat_186,feat_187,feat_188,feat_189,feat_190,feat_191,feat_192,feat_193,feat_194,feat_195,feat_196,feat_197,feat_198,feat_199,feat_200,feat_201,feat_202,feat_203,feat_204,feat_205,feat_206,feat_207,feat_208,feat_209,feat_210,feat_211,feat_212,feat_213,feat_214,feat_215,feat_216,feat_217,feat_218,feat_219,feat_220,feat_221,feat_222,feat_223,feat_224,feat_225,feat_226,feat_227,feat_228,feat_229,feat_230,feat_231,feat_232,feat_233,feat_234,feat_235,feat_236,feat_237,feat_238,feat_239,feat_240,feat_241,feat_242,feat_243,feat_244,feat_245,feat_246,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,feat_257,feat_258,feat_259,feat_260,feat_261,feat_262,feat_263,feat_264,feat_265,feat_266,feat_267,feat_268,feat_269,feat_270,feat_271,feat_272,feat_273,feat_274,feat_275,feat_276,feat_277,feat_278,feat_279,feat_280,feat_281,feat_282,feat_283,feat_284,feat_285,feat_286,feat_287,feat_288,feat_289,feat_290,feat_291,feat_292,feat_293,feat_294,feat_295,feat_296,feat_297,feat_298,feat_299,feat_300,feat_301,feat_302,feat_303,feat_304,feat_305,feat_306,feat_307,feat_308,feat_309,feat_310,feat_311
0,0,7340,"[0.10458118, 0.0...",0.104581,0.047880,0.030944,-0.035116,-0.026452,-0.018017,0.016048,-0.045419,0.000311,0.093888,0.068062,-0.041069,0.062561,0.001897,-0.063457,-0.005675,0.015704,-0.061965,0.014970,-0.024888,0.056331,0.014982,0.047225,0.021182,0.079994,-0.021951,-0.057099,0.016034,-0.030449,0.024281,0.010529,-0.023831,0.011418,-0.067547,-0.024365,0.052528,-0.020069,0.004901,-0.036743,0.003144,-0.018055,0.248296,0.073590,0.014884,-0.042709,0.004142,-0.009092,-0.006609,0.015600,-0.007840,0.043871,-0.041044,0.020118,-0.052628,0.077389,-0.001171,0.018893,-0.041031,0.031918,-0.038071,0.003181,0.080917,-0.037467,0.020274,-0.024794,0.140124,0.085882,-0.037249,0.022786,0.001536,0.086229,-0.075313,-0.031434,-0.020224,0.009186,0.015094,0.061660,-0.044022,0.064016,0.019537,-0.034990,0.050758,0.036314,0.056562,0.006112,0.032237,-0.009762,0.030743,-0.014273,-0.009490,0.055373,-0.064723,0.011089,0.035945,-0.002951,0.011540,0.072136,0.028247,-0.043543,0.006763,-0.076490,-0.119404,-0.047220,-0.098469,-0.026963,-0.043232,0.052229,0.063210,0.077692,-0.023310,0.036534,0.002803,-0.058378,0.033859,0.001762,0.025262,-0.060648,0.101775,-0.031662,-0.045905,0.001397,-0.091876,0.014755,-0.011848,-0.019974,0.011138,0.037368,0.034023,0.012566,0.033387,0.046067,0.01894

In [4]:
# candidates содержит item_id свежих кандидатов из которых нужно будет предсказать на тесте
candidates_df = pd.read_parquet('fresh_candidates.parquet.gzip')
print("len rows: ", candidates_df.shape[0])
print("Items for predict: ", candidates_df.item_id.nunique())
candidates_df.head()

len rows:  100000
Items for predict:  100000


,item_id
0,0
1,2
2,5
3,6
4,7


In [5]:
test_id = pd.read_parquet('test.parquet.gzip')
print("len rows: ", test_id.shape[0])
print("Test users: ", test_id.user_id.nunique())
test_id.head()

len rows:  200000
Test users:  200000


,user_id
0,7
1,8
2,9
3,11
4,18


In [8]:
#simple filter
train = train[train.user_id.isin(test_id.user_id)]
train = train[train.item_id.isin(candidates_df.item_id)]

print(train.shape)
train.head()

(16696198, 5)


,time,user_id,item_id,timespent,reaction
4940,4940,863936,72872,4,0
7366,7366,45679,161406,0,0
12790,12790,653187,92984,0,0
13038,13038,890059,72872,3,0
22176,22176,862054,70829,1,0


In [9]:
known_items = collections.defaultdict(list)
max_len = 0

tmp = train[train.item_id.isin(candidates_df.item_id.values)]
tmp = train[train.user_id.isin(test_id.user_id.values)]

for id, row in tqdm(tmp.iterrows(), total=tmp.shape[0]):
    known_items[row.user_id].append(row.item_id)
    l = len(known_items[row.user_id])
    if l > max_len:
        max_len = l

print(max_len)

  0%|          | 0/16696198 [00:00<?, ?it/s]

534


In [7]:
# Item feature matrix
item_feat_map = dict() # норм айди в айди си эс эр матрицы
for i, it_id in enumerate(items_meta.item_id.tolist()):
    item_feat_map[it_id] = i

item_feature_csr = csr_matrix(items_meta[[f"feat_{i}" for i in range(312)]].to_numpy())
item_feature_csr

In [8]:
%%time
# User feature matrix
n_last = 3
user_last_watches = train.groupby("user_id")["item_id"].apply(lambda x: list(x.iloc[-n_last:])).reset_index()
user_last_watches = user_last_watches.explode('item_id')
user_last_watches = user_last_watches.merge(items_meta[["item_id", "embeddings"]], on='item_id')
user_last_watches = pd.DataFrame(user_last_watches.groupby(['user_id'])['embeddings'].agg(np.mean)).reset_index()
user_last_watches[[f"feat_{i}" for i in range(312)]] = user_last_watches.embeddings.parallel_apply(pd.Series)
user_feature_csr = csr_matrix(user_last_watches[[f"feat_{i}" for i in range(312)]].to_numpy())
user_feature_csr

In [9]:
# sparse.save_npz("user_feature_csr_3.npz", user_feature_csr)
user_feature_csr = sparse.load_npz("user_feature_csr_3.npz")

## Обучение LightFM

In [10]:
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

In [11]:
dataset = Dataset()
dataset.fit(train['user_id'].unique(), train['item_id'].unique())

In [12]:
%%time
# matrix for training
interactions_matrix, weights_matrix = dataset.build_interactions(
    zip(*train[['user_id', 'item_id', 'timespent']].values.T)
)

weights_matrix_csr = weights_matrix.tocsr()

CPU times: user 43.4 s, sys: 521 ms, total: 43.9 s
Wall time: 43.9 s


In [13]:
# user / item mappings
lightfm_mapping = dataset.mapping()
lightfm_mapping = {
    'users_mapping': lightfm_mapping[0],
    'items_mapping': lightfm_mapping[2],
}

lightfm_mapping['users_inv_mapping'] = {v: k for k, v in lightfm_mapping['users_mapping'].items()}
lightfm_mapping['items_inv_mapping'] = {v: k for k, v in lightfm_mapping['items_mapping'].items()}

print(f"users_mapping amount: {len(lightfm_mapping['users_mapping'])}")
print(f"items_mapping amount: {len(lightfm_mapping['items_mapping'])}")

users_mapping amount: 199985
items_mapping amount: 99812


In [15]:
lfm_model = LightFM(
    no_components=256, # размерность пространства
    learning_rate=0.01, 
    loss='warp', 
    # item_alpha=0.0, # регуляризация
    # user_alpha=0.0, # регуляризация
    max_sampled=10, # сколько негатив майнить
    random_state=42,
)

In [16]:
#  train model

num_epochs = 10

for _ in tqdm(range(num_epochs)):
    lfm_model.fit_partial(
        interactions=weights_matrix_csr,
        user_features=None, # np.float32 csr_matrix of shape [n_users, n_user_features]
        item_features=None, # np.float32 csr_matrix of shape [n_items, n_item_features]
        # sample_weight=None, # np.float32 coo_matrix of shape [n_users, n_items] по таймспент (учтено в interactions)
        epochs=1,
        num_threads=20,
        # verbose=False,
    )
    # test_precision = precision_at_k(
        # lfm_model, 
        # weights_matrix_csr_test, 
        # train_interactions=weights_matrix_csr, 
        # k=20, 
        # num_threads=20).mean()
    # print(test_precision)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
# save model  
with open(f"lfm_model_itemFEAT_11jan.dill", 'wb') as f:
    dill.dump(lfm_model, f)

## inference

In [17]:
# фильруем нужные айтемы и юзеры из горячих (которые были в обучении)
warm_users = []
cold_users = []
for user in test_id.user_id.tolist():
    if user in lightfm_mapping['users_mapping'].keys():
        warm_users.append(user)
    else:
        cold_users.append(user)

print("warm users: ", len(warm_users))
print("cold users: ", len(cold_users))

warm users:  199985
cold users:  15


In [18]:
warm_items = []
cold_items = []
for item in candidates_df.item_id.tolist():
    if item in lightfm_mapping['items_mapping'].keys():
        warm_items.append(item)
    else:
        cold_items.append(item)

print("Warm items: ", len(warm_items))
print("Cold items: ", len(cold_items))

Warm items:  99812
Cold items:  188


In [19]:
candidates = pd.DataFrame({
    'user_id': warm_users
})

candidates.head(3)

,user_id
0,7
1,8
2,9


In [20]:
def generate_lightfm_recs_mapper(model, item_ids, known_items,
                                 user_features, item_features, N, 
                                 user_mapping, item_inv_mapping, 
                                 num_threads=1):
    def _recs_mapper(user): # обычный юзер
        user_id = user_mapping[user] # псевдо юзер
        # user_id = user
        recs = model.predict(user_id, # Для какого юзера будем рекомендовать
                             item_ids, # массив айтемов из обучения
                             user_features=user_features, # np.float32 csr_matrix of shape [n_users, n_user_features], optional
                             item_features=item_features,  # np.float32 csr_matrix of shape [n_items, n_item_features], optional
                             num_threads=num_threads)
        
        additional_N = len(known_items[user]) if user in known_items else 0
        total_N = N + additional_N
        top_cols = np.argpartition(recs, -np.arange(total_N))[-total_N:][::-1]
        
        final_recs = [item_inv_mapping[pseudo_id_mapper[item]] for item in top_cols]
        if additional_N > 0:
            filter_items = known_items[user]
            final_recs = [item for item in final_recs if item not in filter_items]
        return final_recs[:N]
    return _recs_mapper

# кол-во кандидатов 
top_N = 20

# вспомогательные данные 
# all_cols = list(lightfm_mapping['items_mapping'].values()) # какие айтемы будем рекомендовать

pseudo_item_ids = [lightfm_mapping['items_mapping'][item] for item in warm_items]
pseudo_id_mapper = dict()
for i, item in enumerate(pseudo_item_ids):
    pseudo_id_mapper[i] = item

mapper = generate_lightfm_recs_mapper(
    lfm_model, 
    item_ids=pseudo_item_ids, 
    known_items=known_items, # тут можно добавить айтемы из трейна, чтобы они не повторялись в предсказаниях
    N=top_N,
    user_features=None, 
    item_features=None, 
    user_mapping=lightfm_mapping['users_mapping'],
    item_inv_mapping=lightfm_mapping['items_inv_mapping'],
    num_threads=1
)

In [21]:
%%time
# генерируем предказания
candidates['predictions'] = candidates['user_id'].parallel_map(mapper)

CPU times: user 22min 14s, sys: 1min 55s, total: 24min 10s
Wall time: 1h 21min 24s


In [22]:
candidates

,user_id,predictions
0,7,"[120767, 225411,..."
1,8,"[43635, 210210, ..."
2,9,"[223491, 56204, ..."
3,11,"[168651, 43635, ..."
4,18,"[87797, 120767, ..."
...,...,...
199980,1000160,"[155049, 58272, ..."
199981,1000165,"[158392, 1537, 8..."
199982,1000166,"[14760, 183888, ..."
199983,1000168,"[76464, 117035, ..."


In [23]:
cold_pred = pd.DataFrame({'user_id': cold_users})
most_views = [158392, 1537, 155049, 126834, 151406, 168651, 84951, 106246, 93615, 225171, 23623, 32641, 183888, 205187, 197397, 119238, 221911, 117035, 149746, 43635]
cold_pred['predictions'] = cold_pred.user_id.apply(lambda x: most_views)
cold_pred

,user_id,predictions
0,14910,"[158392, 1537, 1..."
1,31656,"[158392, 1537, 1..."
2,33612,"[158392, 1537, 1..."
3,78730,"[158392, 1537, 1..."
4,88598,"[158392, 1537, 1..."
5,105397,"[158392, 1537, 1..."
6,177374,"[158392, 1537, 1..."
7,234422,"[158392, 1537, 1..."
8,361346,"[158392, 1537, 1..."
9,366677,"[158392, 1537, 1..."


In [24]:
final = pd.concat([cold_pred, candidates])
final

,user_id,predictions
0,14910,"[158392, 1537, 1..."
1,31656,"[158392, 1537, 1..."
2,33612,"[158392, 1537, 1..."
3,78730,"[158392, 1537, 1..."
4,88598,"[158392, 1537, 1..."
...,...,...
199980,1000160,"[155049, 58272, ..."
199981,1000165,"[158392, 1537, 8..."
199982,1000166,"[14760, 183888, ..."
199983,1000168,"[76464, 117035, ..."


In [25]:
final.to_parquet('lightfm_sub_12jan_need.parquet.gzip', compression='gzip')